In [1]:
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [2]:
def OCRclean( a ):
    import string
    import pkg_resources
    from symspellpy import SymSpell, Verbosity
    import re

    word = a.split() #removing white space


    html = [ "&amp;" , "&quot;" , "&apos;" , "&gt;" , "&lt;"]
    symbol = [ "&"  , '"' , "'", ">" , "<"]

    for i in range(4):
        a = re.sub(html[i], symbol[i], a)

    remove = string.punctuation
    remove = remove.replace(",", "")
    remove = remove.replace(".", "")
    remove = remove.replace("?", "")
    remove = remove.replace("!", "")

    table = str.maketrans('', '', remove) #removing punctuations
    stripped = [w.translate(table) for w in word]
    stripped = list(filter(lambda a: a != '', stripped))

    sym_spell = SymSpell(max_dictionary_edit_distance=4, prefix_length=7)
    dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
    
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)



    cleaned = []
    for word in stripped:
        qualify = True

        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=4, include_unknown = True,transfer_casing=True)

        date = { "1st", "2nd", "3rd", "4th", "5th", "6th", "7th", "8th", "9th", "0th"}
        punctuation = { ',' , '.' , '?' , "!"}
        
        if word in date:
            qualify = False
        
        if word in punctuation:
            qualify = False

        if word.isnumeric():
            qualify = False

        if qualify:
           cleaned.append(suggestions[0].term)
        else:
            cleaned.append(word)


    return ' '.join(cleaned)


In [3]:
ad_list = ["490386275.xml", "490379248.xml"] # list of xml file names
#file_name = input("What xml file do you want to add?")
#ad_list.append(file_name)
#print(ad_list)

data = pd.DataFrame(columns = ['ID', 'Publisher', 'Product', "Date", "Name_of_Company", "Location", "Full_Text"])
data.head()

,ID,Publisher,Product,Date,Name_of_Company,Location,Full_Text


In [4]:
for counter, ad in enumerate(ad_list):
    xml = open(ad ,"r")
    content = xml.read()

    soup = bs(content, 'xml') #this solves soup name undefined

    #print(soup.prettify()) # this is the list 

    info_dict = dict()

    id_num = soup.findAll("RecordID") 

    for num in id_num: 
        numbers = (num.get_text())
    info_dict["ID"] = OCRclean(numbers)  

    texts = soup.findAll("FullText" or "Abstract")
    for text in texts:
        a = (text.get_text()) # full text
    info_dict["Full_Text"] = OCRclean(a)

    publishers = soup.findAll("Publisher")
    for publisher in publishers:
        publish = (publisher.get_text())
    info_dict["Publisher"] = OCRclean(publish)

    locations = soup.findAll("Qualifier")
    for location in locations:
        loc = (location.get_text())
    info_dict["Location"] = OCRclean(loc)

    dates = soup.findAll("AlphaPubDate")
    for date in dates:
        da = date.get_text()
    info_dict["Date"] = OCRclean(da)

    titles = soup.findAll("RecordTitle")
    for title in titles:
        ti = title.get_text()
    info_dict["Product"] = OCRclean(ti)

    data = data.append(info_dict, ignore_index = True)
    info_dict = info_dict.fromkeys(info_dict, 0)
 

In [5]:
data

,ID,Publisher,Product,Date,Name_of_Company,Location,Full_Text
0,490386275,Atlanta Daily World,SIMS ALSO TO DIE IN CHAIR,Apr of 1932,NaN,Atlanta Gay,SEll a in To me in LOU I A PIC III Richard Sim...
1,490379248,Atlanta Daily World,CHILD NEAR DEATH DUE TO BURNS,Dec the 1931,NaN,Atlanta Gay,CHILD HEART DEATH DUE TO BURNS Dress Ignites F...


In [6]:
df_new = pd.read_csv('Atlantic_new.csv')
df_new.head(10)
df_new.drop(0, axis = 0)

,RecordID,AlphaPubDate,Publisher,FullText,Abstract
1,490378711.0,"Dec 30, 1931",Atlanta Daily World,PDRa^^Nc. Jj^^^S,NaN
2,490378818.0,"Dec 25, 1931",Atlanta Daily World,IN In memory of our dearly ...,NaN
3,490378993.0,"Dec 18, 1931",Atlanta Daily World,5th Carol Concert oi' Two Colleges Promises Tr...,NaN
4,490379043.0,"Dec 25, 1931",Atlanta Daily World,"Chicago, Dec. 22--(By The Associated Negro Pre...",1005880true1007945true1008888true1009131true10...
5,490379071.0,"Dec 30, 1931",Atlanta Daily World,Wake Up Your Liver Bile -Without Calomel ...,NaN
...,...,...,...,...,...
24996,491176844.0,"Jul 12, 1959",Atlanta Daily World,PER...,NaN
24997,491190839.0,"Jan 12, 1960",Atlanta Daily World,CHI...,1005880true1007945true1008888true1009131true10...
24998,491193319.0,"Feb 18, 1960",Atlanta Daily World,CHI...,1005880true1007945true1008888true1009131true10...
24999,491199289.0,"Apr 16, 1960",Atlanta Daily World,Dr....,1005880true1007945true1008888true1009131true10...


In [ ]:
text = df_new["FullText"]

In [7]:
df_new.shape

(25001, 5)

In [10]:
df_new["FullText"] = df_new["FullText"].astype("str")

In [14]:
df_new["CleanText"] = "Nan"

In [15]:
# adding column to dataset that contains cleaned full texts
for counter, text in enumerate(df_new["FullText"]):
    df_new["CleanText"][counter] = OCRclean(text)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 

In [ ]:
df_new["Product"] = "Nan"

In [ ]:
# NER --> Named Entity Recognition
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [ ]:
# creating function that will give product and input into new dataframe column
def NER(df, fulltext, product_col): # df = name of df, fulltext = column name that has full text, product_col = column name to put product
    products = []
    for number, column in enumerate(df[fulltext]):
        # creates a list of entities
        sent = preprocess(column)
        # finding product in each full text
        for word in sent:
            if "NNP" in word:
                products.append(word)
        # trying to input product into a new column called product
        df[product_col][number] = ", ".join(list(set(products)))


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=df3dab1c-1816-4704-b405-025e9b8250ee' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>